<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Scope" data-toc-modified-id="Scope-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Scope</a></span></li><li><span><a href="#Import-libraries" data-toc-modified-id="Import-libraries-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Import libraries</a></span></li><li><span><a href="#Import-json-file" data-toc-modified-id="Import-json-file-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Import json file</a></span></li><li><span><a href="#Summary" data-toc-modified-id="Summary-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Summary</a></span></li></ul></div>

### Scope
Loop through artists and their song names through attain its lyrics by navigating through the URLs of AZLyrics. Eventually put all 3 information into a dataframe to work with.

### Import libraries

In [178]:
# file types
import csv
import json

# standard inputs
import pandas as pd
import numpy as np

# For web scraping
import requests
from bs4 import BeautifulSoup

# For performing regex operations
import re

# For adding delays so that we don't spam requests
import time

### Import json file

In [200]:
# file containing artists - songs mapping
lyrical_rap_songs_json = "lyrical_rap_songs.json"
lyrical_dict = {}

with open(lyrical_rap_songs_json) as file:
    lyrical_dict = json.load(file)

In [13]:
# Viewing dictionary in a readable manner
# print(json.dumps(lyrical_dict, indent=4,sort_keys=True))

In [201]:
# Viewing names of songs
list(lyrical_dict.values())

[['Intro - Skit',
  "I'm Here",
  'Break It Down',
  'Who Do You Think I Am',
  'Sexy Cyber',
  'Pans In The Kitchen',
  'Story Of Mine',
  'Bubblies',
  'Chris Dolmeth',
  'The B Bop',
  'Slurpin',
  'Super Duper Fly',
  'Motherfucker',
  "Don't Trust 'Em",
  'Gazing At The Moonlight',
  "Hot 16's",
  'Sag My Pants',
  'You Are My Enemy',
  'Trampoline',
  "I'm Not Introducing You",
  "I Can't Decide",
  'I Am RAW',
  'Nocturnal Rainbows',
  'How You Like Me Now',
  'Heather Nicole',
  'Kill Her',
  "I'm Not Crazy",
  'Where Will I Go',
  "Baby's Daddy",
  'Blood Energy Potion',
  'Pillow Man',
  'Fiends Are Knocking',
  'Hop Is Back',
  "Who's There?",
  'Tears To Snow',
  'Rip Your Heart Out',
  'Nollie Tre Flip',
  'Give Me That Money',
  'I Need Help',
  'Hip Hop Sinister',
  'Good Guys Get Left Behind',
  'Bad Manners Freestyle',
  'Ill Mind Of Hopsin 6: Old Friend',
  'Still Got Love For You',
  'Jungle Bash',
  'Lunchtime Cypher',
  'Dream Forever',
  "What's My Purpose?",
  'C

In [202]:
len(list(lyrical_dict.values()))

37

In [203]:
print(lyrical_dict.keys())

dict_keys(['Hopsin', 'NF (Nathan Feuerstein)', 'T.I.', 'Anderson .Paak', 'Andre 3000', 'Big L', 'Big Sean', 'Black Thought', 'Busta Rhymes', 'Chance The Rapper', 'YBN Cordae (Entendre)', 'Denzel Curry', 'Eminem', 'Ice Cube', 'Jay-Z', 'J. Cole', 'Joyner Lucas', 'Kendrick Lamar', 'Lil Dicky', 'Lil Wayne', 'Logic', 'Macklemore', 'Marlon Craft', 'Meek Mill (Meek Millz)', 'Method Man', 'MF Doom', 'Mos Def (Yasiin Bey)', 'Nas', 'Nate Dogg', 'Nipsey Hussle', 'Notorious B.I.G. (Biggie Smalls)', 'Pusha T', 'Rakim', 'Royce Da 5\'9"', 'Snoop Dogg', 'Token', 'Wale'])


The dictionary's keys are the names of the artists while the values are the names of their respective songs. There happens to be 37 artists.

Will need to tidy up the names (dict keys) and their respective song titles (to only contain letters and numbers) so it'll be easier to scrape data via url. 

In [204]:
# lyrical rap artist list
lyrical_artists = ["Hopsin","NF","TI","anderson-paak",
                   "andre3000","bigl","bigsean","blackthought","busta",
                   "chancetherapper","YBNCordae","denzelcurry",
                   "eminem","icecube","jayz","jcole","joynerlucas",
                   "kendricklamar","lildicky","lilwayne","logic",
                   "macklemore","marloncraft","meekmill","methodman",
                   "mfdoom","mosdef","nas","natedogg","nipseyhussle",
                   "notorious","pushat","rakim","royceda59",
                   "snoopdogg","token","wale"]

# lyrical_artists = ["Hopsin"]

# changing keys of dictionary to formatting for link
lyrical_dict = dict(zip(lyrical_artists, list(lyrical_dict.values())))

# Check
lyrical_dict.keys()

dict_keys(['Hopsin', 'NF', 'TI', 'anderson-paak', 'andre3000', 'bigl', 'bigsean', 'blackthought', 'busta', 'chancetherapper', 'YBNCordae', 'denzelcurry', 'eminem', 'icecube', 'jayz', 'jcole', 'joynerlucas', 'kendricklamar', 'lildicky', 'lilwayne', 'logic', 'macklemore', 'marloncraft', 'meekmill', 'methodman', 'mfdoom', 'mosdef', 'nas', 'natedogg', 'nipseyhussle', 'notorious', 'pushat', 'rakim', 'royceda59', 'snoopdogg', 'token', 'wale'])

The dictionary's keys seem to have the right url format. Just to tidy up the song names now.

In [205]:
for key in lyrical_dict:
    
    lyrical_rap_songs = []
    
    for x in lyrical_dict[key]:
        x = x.replace(" ", "").lower()
        x = x.replace("(", "")
        x = x.replace(")", "")
        x = x.replace(".", "")
        x = x.replace(",", "")
        x = x.replace("'", "")
        x = x.replace("/", "")
        x = x.replace("?", "")
        x = x.replace("#", "")
        x = x.replace("$", "s")
        x = x.replace("&", "and")
        x = x.replace(":", "")
        x = x.replace("-", "")
        
        # if statement to prevent duplicate song names from being added
        if x not in lyrical_rap_songs:
                lyrical_rap_songs.append(x)
            
        
    lyrical_dict[key] = lyrical_rap_songs

In [24]:
# Check
lyrical_dict

{'Hopsin': ['introskit',
  'imhere',
  'breakitdown',
  'whodoyouthinkiam',
  'sexycyber',
  'pansinthekitchen',
  'storyofmine',
  'bubblies',
  'chrisdolmeth',
  'thebbop',
  'slurpin',
  'superduperfly',
  'motherfucker',
  'donttrustem',
  'gazingatthemoonlight',
  'hot16s',
  'sagmypants',
  'youaremyenemy',
  'trampoline',
  'imnotintroducingyou',
  'icantdecide',
  'iamraw',
  'nocturnalrainbows',
  'howyoulikemenow',
  'heathernicole',
  'killher',
  'imnotcrazy',
  'wherewilligo',
  'babysdaddy',
  'bloodenergypotion',
  'pillowman',
  'fiendsareknocking',
  'hopisback',
  'whosthere',
  'tearstosnow',
  'ripyourheartout',
  'nollietreflip',
  'givemethatmoney',
  'ineedhelp',
  'hiphopsinister',
  'goodguysgetleftbehind',
  'badmannersfreestyle',
  'illmindofhopsin6oldfriend',
  'stillgotloveforyou',
  'junglebash',
  'lunchtimecypher',
  'dreamforever',
  'whatsmypurpose',
  'caughtintherain',
  'thepoundintro',
  'foreveriii',
  'nohope',
  'ramona',
  'mrjones',
  'fortcol

In [206]:
len(lyrical_dict.items())

37

Can confirm that there are 37 artists and their songs seem to be well alligned with the url's format: azlyrics.com/lyrics/{1}/{2}.html

Where {1} is tyhe artist's name and {2} is the song's name.

In [207]:
# url to scrape the lyrics from
base_url = "https://www.azlyrics.com/lyrics/{}/{}.html"

In [208]:
# creating the empty dictionary for artists, songs, and lyrics
lyrical_data_dict = {'lyrics': [], 'song': [], 'artist': []}
count=0

# looping through each artist
for artists in lyrical_dict:
    print(f"\n started {count} {artists}")
    
    # looping through each song
    for songs in lyrical_dict[artists]:
        print(f"\n started {songs}")
        
        dummy_var = 0
        
        # looping url
        final_url = base_url.format(artists.lower(), songs.lower())

        print(f"updated {final_url}")

        time.sleep(2)

        while dummy_var == 0:

            response = requests.get(final_url)

            print("got response")
            
            soup = BeautifulSoup(response.content)
            
            print("got soup")

            running = soup.find('div', class_="col-xs-12 col-lg-8 text-center")
            
            print("finished running")
            
            time.sleep(2)
            
            dummy_var=1
        
        try: 
            print("converting into text")
            running= running.text

            running_cleaned = re.sub("\n\n\n\n", "", running)

            # phrase before the first lyric - indication
            start1 = running_cleaned.find("Lyrics")

            # phrase after the last lyric - indication
            end1 = running_cleaned.find("Submit Corrections")

            running_cleaned = running_cleaned[start1:end1]

            start2 = running_cleaned.find('\n\n')

            # snipping
            running_cleaned = running_cleaned[start2:-1]

            running_cleaned = re.sub("\n\n\n\r\n","", running_cleaned)

            lyrical_data_dict["lyrics"].append(running_cleaned)
            lyrical_data_dict["song"].append(songs)
            lyrical_data_dict["artist"].append(artists)

            print(f"Processed lyrics for {final_url}")

            # 3 seconds b/w each iteration to prevent spam
            time.sleep(2)
        except:
            lyrical_data_dict["lyrics"].append("error")
            print("error with lyrics")
            lyrical_data_dict["song"].append(songs)
            lyrical_data_dict["artist"].append(artists)
            
    count=count+1
    
    print(f"finished {artist}")
          
    print(f"{count}, {artist}")


 started 0 Hopsin

 started introskit
updated https://www.azlyrics.com/lyrics/hopsin/introskit.html
got response
got soup
finished running
converting into text
Processed lyrics for https://www.azlyrics.com/lyrics/hopsin/introskit.html

 started imhere
updated https://www.azlyrics.com/lyrics/hopsin/imhere.html
got response
got soup
finished running
converting into text
Processed lyrics for https://www.azlyrics.com/lyrics/hopsin/imhere.html

 started breakitdown
updated https://www.azlyrics.com/lyrics/hopsin/breakitdown.html
got response
got soup
finished running
converting into text
Processed lyrics for https://www.azlyrics.com/lyrics/hopsin/breakitdown.html

 started whodoyouthinkiam
updated https://www.azlyrics.com/lyrics/hopsin/whodoyouthinkiam.html
got response
got soup
finished running
converting into text
Processed lyrics for https://www.azlyrics.com/lyrics/hopsin/whodoyouthinkiam.html

 started sexycyber
updated https://www.azlyrics.com/lyrics/hopsin/sexycyber.html
got response



 started givemethatmoney
updated https://www.azlyrics.com/lyrics/hopsin/givemethatmoney.html
got response
got soup
finished running
converting into text
Processed lyrics for https://www.azlyrics.com/lyrics/hopsin/givemethatmoney.html

 started ineedhelp
updated https://www.azlyrics.com/lyrics/hopsin/ineedhelp.html
got response
got soup
finished running
converting into text
Processed lyrics for https://www.azlyrics.com/lyrics/hopsin/ineedhelp.html

 started hiphopsinister
updated https://www.azlyrics.com/lyrics/hopsin/hiphopsinister.html
got response
got soup
finished running
converting into text
Processed lyrics for https://www.azlyrics.com/lyrics/hopsin/hiphopsinister.html

 started goodguysgetleftbehind
updated https://www.azlyrics.com/lyrics/hopsin/goodguysgetleftbehind.html
got response
got soup
finished running
converting into text
Processed lyrics for https://www.azlyrics.com/lyrics/hopsin/goodguysgetleftbehind.html

 started badmannersfreestyle
updated https://www.azlyrics.com/

got response
got soup
finished running
converting into text
Processed lyrics for https://www.azlyrics.com/lyrics/hopsin/thepurge.html

 started happyending
updated https://www.azlyrics.com/lyrics/hopsin/happyending.html
got response
got soup
finished running
converting into text
Processed lyrics for https://www.azlyrics.com/lyrics/hopsin/happyending.html

 started nowords2skit
updated https://www.azlyrics.com/lyrics/hopsin/nowords2skit.html
got response
got soup
finished running
converting into text
Processed lyrics for https://www.azlyrics.com/lyrics/hopsin/nowords2skit.html

 started panoramacity
updated https://www.azlyrics.com/lyrics/hopsin/panoramacity.html
got response
got soup
finished running
converting into text
Processed lyrics for https://www.azlyrics.com/lyrics/hopsin/panoramacity.html

 started illmindofhopsin9
updated https://www.azlyrics.com/lyrics/hopsin/illmindofhopsin9.html
got response
got soup
finished running
converting into text
Processed lyrics for https://www.az

got response
got soup
finished running
converting into text
error with lyrics

 started sounfair
updated https://www.azlyrics.com/lyrics/hopsin/sounfair.html
got response
got soup
finished running
converting into text
error with lyrics

 started theoldus
updated https://www.azlyrics.com/lyrics/hopsin/theoldus.html
got response
got soup
finished running
converting into text
error with lyrics

 started traceysstalker
updated https://www.azlyrics.com/lyrics/hopsin/traceysstalker.html
got response
got soup
finished running
converting into text
error with lyrics

 started westcoasteclipse
updated https://www.azlyrics.com/lyrics/hopsin/westcoasteclipse.html
got response
got soup
finished running
converting into text
error with lyrics

 started westside
updated https://www.azlyrics.com/lyrics/hopsin/westside.html
got response
got soup
finished running
converting into text
error with lyrics

 started youshouldveknown
updated https://www.azlyrics.com/lyrics/hopsin/youshouldveknown.html
got resp

got response
got soup
finished running
converting into text
error with lyrics

 started icanfeelit
updated https://www.azlyrics.com/lyrics/nf/icanfeelit.html
got response
got soup
finished running
converting into text
error with lyrics

 started gotyouonmymind
updated https://www.azlyrics.com/lyrics/nf/gotyouonmymind.html
got response
got soup
finished running
converting into text
error with lyrics

 started grindin
updated https://www.azlyrics.com/lyrics/nf/grindin.html
got response
got soup
finished running
converting into text
error with lyrics

 started wishyouwouldnt
updated https://www.azlyrics.com/lyrics/nf/wishyouwouldnt.html
got response
got soup
finished running
converting into text
error with lyrics

 started statement
updated https://www.azlyrics.com/lyrics/nf/statement.html
got response
got soup
finished running
converting into text
error with lyrics

 started allido
updated https://www.azlyrics.com/lyrics/nf/allido.html
got response
got soup
finished running
converting in

...

got response
got soup
finished running
converting into text
error with lyrics

 started nodaysoff
updated https://www.azlyrics.com/lyrics/wale/nodaysoff.html
got response
got soup
finished running
converting into text
error with lyrics

 started dcornothing
updated https://www.azlyrics.com/lyrics/wale/dcornothing.html
got response
got soup
finished running
converting into text
error with lyrics

 started thatway
updated https://www.azlyrics.com/lyrics/wale/thatway.html
got response
got soup
finished running
converting into text
error with lyrics

 started theforward
updated https://www.azlyrics.com/lyrics/wale/theforward.html
got response
got soup
finished running
converting into text
error with lyrics

 started changeup
updated https://www.azlyrics.com/lyrics/wale/changeup.html
got response
got soup
finished running
converting into text
error with lyrics

 started theshow
updated https://www.azlyrics.com/lyrics/wale/theshow.html
got response
got soup
finished running
converting into t

converting into text
error with lyrics

 started chess
updated https://www.azlyrics.com/lyrics/wale/chess.html
got response
got soup
finished running
converting into text
error with lyrics

 started juggin
updated https://www.azlyrics.com/lyrics/wale/juggin.html
got response
got soup
finished running
converting into text
error with lyrics

 started outronarcissisherspaceshipfreestyle
updated https://www.azlyrics.com/lyrics/wale/outronarcissisherspaceshipfreestyle.html
got response
got soup
finished running
converting into text
error with lyrics

 started theintroaboutnothing
updated https://www.azlyrics.com/lyrics/wale/theintroaboutnothing.html
got response
got soup
finished running
converting into text
error with lyrics

 started theheliumballoon
updated https://www.azlyrics.com/lyrics/wale/theheliumballoon.html
got response
got soup
finished running
converting into text
error with lyrics

 started thewhiteshoes
updated https://www.azlyrics.com/lyrics/wale/thewhiteshoes.html
got respo

got response
got soup
finished running
converting into text
error with lyrics

 started smile
updated https://www.azlyrics.com/lyrics/wale/smile.html
got response
got soup
finished running
converting into text
error with lyrics

 started itscomplicated
updated https://www.azlyrics.com/lyrics/wale/itscomplicated.html
got response
got soup
finished running
converting into text
error with lyrics

 started blackbonnie
updated https://www.azlyrics.com/lyrics/wale/blackbonnie.html
got response
got soup
finished running
converting into text
error with lyrics

 started effortless
updated https://www.azlyrics.com/lyrics/wale/effortless.html
got response
got soup
finished running
converting into text
error with lyrics

 started letitgo
updated https://www.azlyrics.com/lyrics/wale/letitgo.html
got response
got soup
finished running
converting into text
error with lyrics

 started negotiations
updated https://www.azlyrics.com/lyrics/wale/negotiations.html
got response
got soup
finished running
con

got response
got soup
finished running
converting into text
error with lyrics

 started 4am
updated https://www.azlyrics.com/lyrics/wale/4am.html
got response
got soup
finished running
converting into text
error with lyrics

 started 600benz
updated https://www.azlyrics.com/lyrics/wale/600benz.html
got response
got soup
finished running
converting into text
error with lyrics

 started allmylove
updated https://www.azlyrics.com/lyrics/wale/allmylove.html
got response
got soup
finished running
converting into text
error with lyrics

 started allstarbreakup
updated https://www.azlyrics.com/lyrics/wale/allstarbreakup.html
got response
got soup
finished running
converting into text
error with lyrics

 started arrival
updated https://www.azlyrics.com/lyrics/wale/arrival.html
got response
got soup
finished running
converting into text
error with lyrics

 started astonmartinmusicfreestyle
updated https://www.azlyrics.com/lyrics/wale/astonmartinmusicfreestyle.html
got response
got soup
finished

got response
got soup
finished running
converting into text
error with lyrics

 started losingfocus
updated https://www.azlyrics.com/lyrics/wale/losingfocus.html
got response
got soup
finished running
converting into text
error with lyrics

 started makeitrainfreestyle
updated https://www.azlyrics.com/lyrics/wale/makeitrainfreestyle.html
got response
got soup
finished running
converting into text
error with lyrics

 started massappealfreestyle
updated https://www.azlyrics.com/lyrics/wale/massappealfreestyle.html
got response
got soup
finished running
converting into text
error with lyrics

 started miketomlinblackandyellowfreestyle
updated https://www.azlyrics.com/lyrics/wale/miketomlinblackandyellowfreestyle.html
got response
got soup
finished running
converting into text
error with lyrics

 started mmgundergod
updated https://www.azlyrics.com/lyrics/wale/mmgundergod.html
got response
got soup
finished running
converting into text
error with lyrics

 started mvpbryceharper
updated htt

In [211]:
# assigning the dictionary to a dataframe
lyrical_rap_lyrics_df = pd.DataFrame({'lyrics':lyrical_data_dict['lyrics'], 'song':lyrical_data_dict['song'], 'artist':lyrical_data_dict['artist']})

In [212]:
lyrical_rap_lyrics_df.shape

(7187, 3)

In [226]:
sum(lyrical_rap_lyrics_df["lyrics"]=="error")

6987

In [229]:
(sum(lyrical_rap_lyrics_df["lyrics"]=="error")/(lyrical_rap_lyrics_df["lyrics"].count()))*100

97.21719771810213

### Summary
97.2% of the extracted data hasn't been extracted properly due to the name of the song used in the url is off by one letter. I'd definitely have to add some elbow grease and fine tune my scraping method. However, I just discovered an API for [Genius](https://genius.com). Will be giving that a go instead.

Not an overall failure though. Got my hands dirty with Regex, BeautifulSoup and json. 